In [1]:
from tool import *

In [15]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    ins_df = pickle.load(f)
with open('D:/Data/nicepeople_cost_db.bin', 'rb') as f:
    nice_df = pickle.load(f)

#### - 월별 업체별 비용 현황

In [16]:
nice_df['Year'] = nice_df['Inspection Date'].dt.year
nice_df['Month'] = nice_df['Inspection Date'].dt.month
df = nice_df.pivot_table('Amount_with_VAT', aggfunc=sum, index='Supplier', columns=['Year', 'Month'])
df.loc['Sum', :] = df.sum()
df.stack(level=0).fillna(0).convert_dtypes()

1         2         3        4        5        6   \
Supplier Year                                                            
Pegatron 2022    136466   2695204    955262   409398        0   136466   
         2023         0         0    740086        0        0        0   
Quanta   2022  13646600  20845181  23045693  3548116  5868038  2865786   
         2023   3848444   2368274   6660770        0        0        0   
Sum      2022  13783066  23540385  24000955  3957514  5868038  3002252   
         2023   3848444   2368274   7400856        0        0        0   

                    7        8        9         10       11      12  
Supplier Year                                                        
Pegatron 2022   136466  1228194  1364660  14055998        0       0  
         2023        0        0        0         0        0       0  
Quanta   2022  2046990  1364660   955262    272932  1910524  545864  
         2023        0        0        0         0        0       0  
Sum      2022  2183456  2592854  2319922  14328930  1910524  545864  
         2023        0        0        0         0        0       0

# 1. 전수검사 DB에서 특정 기간 검사한 내용 조회하기 위한 검색 조건 설정

In [17]:
search_year = '2023'
search_month = list(range(1, 13))
search_date = '.'
sn = '.'
model_name = '.'
responsibility = '.'
reason = '.'
supplier = 'Pegatron'

In [18]:
cond1 = ins_df['Inspection Date'].dt.year.astype(str).str.contains(search_year, na=True)
cond2 = ins_df['Inspection Date'].dt.month.isin(search_month)
cond3 = ins_df['Inspection Date'].dt.day.astype(str).str.contains(search_date, na=True)
cond4 = ins_df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find).str.contains(supplier, na=True)
cond5 = ins_df['Mapping Model.Suffix'].str.contains(model_name, na=True)
cond6 = ins_df['S/N'].str.contains(sn, na=True)
cond7 = ins_df['Responsibility'].str.contains(responsibility, na=True)
cond8 = ins_df['Inspection_Reason'].str.contains(reason, na=True)
inspection_data = ins_df[ cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].copy()

In [19]:
inspection_summary = inspection_data.copy()
inspection_summary['Series'] = inspection_summary['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
inspection_summary['Vendor'] = inspection_summary['Series'].replace(vendor_find)
inspection_summary['Month'] = inspection_summary['Inspection Date'].dt.month
inspection_summary = pd.pivot_table(inspection_summary, index=['Vendor', 'Month', 'Inspection Date','Series', 'Inspection_Reason', 'Responsibility'], 
                        columns=['Judge'], values='S/N', aggfunc='count')
inspection_summary = inspection_summary.fillna(0)
inspection_summary = inspection_summary.convert_dtypes()
inspection_summary['Total'] = inspection_summary['NG'] + inspection_summary['OK']
inspection_summary.loc[:, 'Defective Rate(%)'] = inspection_summary['NG'] * 100 / inspection_summary['Total']
inspection_summary

,,,,,,NG,OK,Total,Defective Rate(%)
Vendor,Month,Inspection Date,Series,Inspection_Reason,Responsibility,,,,
Pegatron,3,2023-03-30,17U70Q,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))",Supplier,10,415,425,2.352941


# 2. 나이스피플 검사비용 DB의 검색조건 설정

In [20]:
fx_2022 = {1: 1193.5, 2: 1204.5, 3: 1207.5, 4: 1220.0, 5: 1248.0, 6: 1298.0, 7: 1305.0, 8: 1358.5, 9: 1428.0, 10: 1421.0, 11: 1309.0, 12: 1273.0}
fx_2023 = {1:1273.0, 2:1226.0, 3:1315.0} # 하나은행 일자별 환율 정보(매매기준율) 참조, 해당 재작업월의 첫번째 날로 조회함
fx = fx_2023

In [21]:
cond1 = nice_df['Inspection Date'].dt.year.astype(str).str.contains(search_year, na=True)
cond2 = nice_df['Inspection Date'].dt.month.isin(search_month)
cond3 = nice_df['Supplier'].str.contains(supplier) 

In [22]:
rework_cost_data = nice_df[cond1 & cond2 & cond3].copy()
rework_cost_data = pd.pivot_table(rework_cost_data, index=['Supplier', 'Inspection Date'], values='Amount_with_VAT', aggfunc=sum)
rework_cost_data = rework_cost_data.reset_index()
rework_cost_data.insert(1, 'Quarter', rework_cost_data['Inspection Date'].dt.quarter)
rework_cost_data.insert(2, 'Month', rework_cost_data['Inspection Date'].dt.month)
rework_cost_data['FX'] = rework_cost_data['Inspection Date'].dt.month.replace(fx)
rework_cost_data['Converted_USD'] = rework_cost_data['Amount_with_VAT'] / rework_cost_data['FX']
rework_cost_data

,Supplier,Quarter,Month,Inspection Date,Amount_with_VAT,FX,Converted_USD
0,Pegatron,1,3,2023-03-30,740086,1315.0,562.803042


In [10]:
rework_cost_data = rework_cost_data.drop(2).reset_index(drop=True) # 나이스피플 재작업 비용 데이터 중, quanta로 청구할 재작업이 아닌 것은 제외함(스타일러스 Pen 배터리 투입 재작업)

In [23]:
date_grp_from_rework_cost = rework_cost_data.groupby('Inspection Date')
date_grp_from_inspection_data = inspection_summary.reset_index().groupby('Inspection Date')

for cost_for_each_day in date_grp_from_rework_cost:
    sub_df = date_grp_from_inspection_data.get_group(cost_for_each_day[0])
    ttl = sub_df['Total'].sum()
    lg = sub_df.loc[sub_df['Responsibility'] == 'LG', 'Total'].sum()
    odm = sub_df.loc[sub_df['Responsibility'] == 'Supplier', 'Total'].sum()
    rework_cost_data.loc[(rework_cost_data['Inspection Date'] == cost_for_each_day[0]), 'Total_Inspected'] = ttl
    rework_cost_data.loc[(rework_cost_data['Inspection Date'] == cost_for_each_day[0]), 'LG_Responsible'] = lg
    rework_cost_data.loc[(rework_cost_data['Inspection Date'] == cost_for_each_day[0], 'Supplier_Responsible')] = odm

rework_cost_data['Amount_LG'] = rework_cost_data['Converted_USD'] * rework_cost_data['LG_Responsible'] / (rework_cost_data['LG_Responsible'] + rework_cost_data['Supplier_Responsible'])
rework_cost_data['Amount_Supplier'] = rework_cost_data['Converted_USD'] * rework_cost_data['Supplier_Responsible'] / (rework_cost_data['LG_Responsible'] + rework_cost_data['Supplier_Responsible']) 
rework_cost_data

,Supplier,Quarter,Month,Inspection Date,Amount_with_VAT,FX,Converted_USD,Total_Inspected,LG_Responsible,Supplier_Responsible,Amount_LG,Amount_Supplier
0,Pegatron,1,3,2023-03-30,740086,1315.0,562.803042,425.0,0.0,425.0,0.0,562.803042


In [24]:
rework_cost_summary = pd.pivot_table(rework_cost_data, index='Supplier', columns=['Month'], values=['Converted_USD', 'Amount_Supplier', 'Amount_LG'], aggfunc=sum).stack(0)
rework_cost_summary.index = rework_cost_summary.index.set_levels(['Amount_LG', 'Amount_Supplier', 'Total'], level=1)
rework_cost_summary

Month                              3
Supplier                            
Pegatron Amount_LG          0.000000
         Amount_Supplier  562.803042
         Total            562.803042

In [25]:
# 참고) 월별 재작업 비용(원화 기준)
rework_cost_KRW = pd.pivot_table(rework_cost_data, index='Supplier', columns=['Quarter', 'Month'], values='Amount_with_VAT', aggfunc=sum)
rework_cost_KRW

Quarter,1
Month,3
Supplier,
Pegatron,740086


In [58]:
rework_cost_KRW.to_clipboard()

In [26]:
filename = f'{datetime.date.today().strftime("%Y%m%d")}_{supplier}_rework_cost_status.xlsx'

In [27]:
with pd.ExcelWriter("D:/재작업비용/"+filename) as writer:
    rework_cost_summary.to_excel(writer, sheet_name='Cost_summary', index=True)
    rework_cost_data.to_excel(writer, sheet_name='rework_cost_data', index=False)
    inspection_summary.to_excel(writer, merge_cells=True, sheet_name='Inspection_Summary', index=True)
    inspection_data.to_excel(writer, merge_cells=True, sheet_name='Inspection_Raw_Data', index=False)